In [ ]:
import re

with open('/content/Interview_Claims.txt', 'r', encoding='utf-8') as file:
    text = file.read()
claims = re.split(r'(?<=[.!?])\s+', text.strip())
claims = [sentence.strip() for sentence in claims if sentence.strip()]


In [ ]:
from openai import OpenAI
import os
import json
import time

openai = OpenAI(api_key="Enter Key")

file_resp = openai.files.create(
    file=open("/content/Participant.pdf", "rb"),
    purpose="assistants"
)

file_id = file_resp.id
print("File uploaded. ID:", file_id)


In [ ]:
import pandas as pd
import io
rows = []


prompt="""You are given:
  - A PDF file containing an interview transcript of a participant.
  Your task:
  1. Carefully analyze the participant’s personality, worldview, tone, and overall patterns of responses from the interview transcript. Identify recurring themes, values, and tendencies that shape their stance on issues.
  2. For the claim given to you, predict whether the participant would likely Support or Oppose or No Clear Evidence for the claim — even if they did not explicitly address it.
  3. Provide an evidence-based justification for your prediction. Connect your reasoning to:
  - Recurring themes and values in the interview,
  - Their personality traits (e.g., optimism, skepticism, openness),
  - Signaling terms or tone that reveal their stance.
  Output:
  Return a JSON object with the following keys:
  {
    "Claim": "original claim text",
    "Stance": "Support / Oppose / No Clear Evidence",
    "Evidence": "Justification connecting prediction to participant’s interview traits, worldview, tone, and signaling terms"
  } \n
 """
for claim in claims:
  
  
  question = prompt + "Given Claim: " + claim

  #question = prompt + "Given Claim : Some participants are unsure what values theoreticians hold."

  resp = openai.responses.create(
      model="gpt-4.1",   # or the correct model / deployment name
      input=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "input_file",
                      "file_id": file_id
                  },
                  {
                      "type": "input_text",
                      "text": question
                  }
              ]
          }
      ]
  )

  print("Response:")
  print(resp.output_text)
  response_text = resp.output_text.strip()
  if response_text.startswith("```json"):
    response_text = response_text.lstrip("```json").strip()
  if response_text.startswith("```"):
    response_text = response_text.lstrip("```").strip()
  if response_text.endswith("```"):
    response_text = response_text.rstrip("```").strip()
  response_text = re.sub(r',(\s*[}\]])', r'\1', response_text)

  result = json.loads(response_text)
  rows.append(result)
  



df = pd.DataFrame(rows)


df.to_csv("Participant.csv", index=False)

print("Saved to Participant.csv")
